In [ ]:
import math
import numpy as np
import random
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
class holographic_reduced_representations:
    
    # Lenght of hrrs
    def __init__(self, length):
        self.length = length
        
    # Creating hrrs
    def hrr(self, normalized=False):
        if normalized:
            x = np.random.uniform(-np.pi,np.pi,int((self.length-1)/2))
            if self.length % 2:
                x = np.real(np.fft.ifft(np.concatenate([np.ones(1), np.exp(1j*x), np.exp(-1j*x[::-1])])))
            else:
                x = np.real(np.fft.ifft(np.concatenate([np.ones(1), np.exp(1j*x), np.ones(1), np.exp(-1j*x[::-1])])))
        else:
            x = np.random.normal(0.0, 1.0/np.sqrt(self.length), self.length)
        return x
    
    # Creates an array of hrrs of size size_of_array x size_of_hrrs
    def hrr_array(self, size_of_array, normalized=False):
        hrr_arr = np.zeros([size_of_array, self.length])
        for x in range(size_of_array):
            hrr_arr[x] = self.hrr(normalized)
        return hrr_arr
    
    # Convolve two hrrs
    def convolve(self, x, y):
        return np.real(np.fft.ifft(np.fft.fft(x)*np.fft.fft(y)))

    # Preconvolve all hrrs to save time
    def preconvolve(self, possible_wm, possible_signals, state_hrrs):
        preconvolved_matrix = np.zeros([possible_wm.size, possible_signals.size, state_hrrs.size, self.length])
        for x in range(len(possible_wm)):
            for y in range(len(possible_signals)):
                for z in range(len(state_hrrs)):
                    preconvolved_matrix[x][y][z] = self.convolve(self.convolve(possible_wm[x], possible_signals[y]), state_hrrs[z])
        return preconvolved_matrix

In [ ]:
class NeuralNetwork:
    def __init__(self, inputs, outputs, discount, alpha):
        self.discount = discount
        self.alpha = alpha
        self.input = inputs
        self.output = outputs
        self.weights = np.random.randn(outputs, inputs)
        self.bias = 1
        
    def feedforward(self, X):
        self.output = np.dot(self.weights, X) + self.bias
        return self.output
    
    def backprop(self, state_prime_hrr, state_hrr, y):
        if(y == reward_good):
            pass
        else:
            pass

In [ ]:
# This has to be followed: self.non_obs_task_switch / self.obs_task_switch = self.num_of_obs_tasks
class maze:
    # Size of the maze
    def __init__(self, size_of_maze, num_of_goals, non_obs_task_switch_rate, num_of_non_obs_tasks, obs_task_switch_rate, num_of_obs_tasks):
        self.size = size_of_maze
        
        # Non observable task switching variables
        self.non_obs_task_switch = non_obs_task_switch_rate
        self.num_of_non_obs_tasks = num_of_non_obs_tasks
        self.non_obs_count = 0
        self.switch_non_obs = 0
        
        # Observable task switching variables
        self.obs_task_switch = obs_task_switch_rate
        self.num_of_obs_tasks = num_of_obs_tasks
        self.obs_count = 0
        self.switch_obs = 0
        
        self.goals = random.sample(range(0, self.size), num_of_goals)  
     
    # Non observable task switching for any number of switches
    def non_obs_switch_func(self):
        if self.non_obs_count == self.non_obs_task_switch:
            self.non_obs_count = 0
            self.switch_non_obs =  self.switch_non_obs + 1
            if self.switch_non_obs >= self.num_of_non_obs_tasks:
                self.switch_non_obs = 0
        self.non_obs_count += 1
        return self.switch_non_obs, self.obs_task_switch_func()
    
    def obs_task_switch_func(self):
        if self.obs_count == self.obs_task_switch:
            self.obs_count = 0
            self.switch_obs = self.switch_obs + 1
            if self.switch_obs >= self.num_of_obs_tasks:
                self.switch_obs = 0
        self.obs_count += 1
        return self.switch_obs

In [ ]:
class agent:
    # Gets the left and right spots
    def get_moves(self, state):
        if(state == 0):
            return self.size - 1, 1
        elif(state == self.size - 1):
            return state - 1, 0
        else:
            return state - 1, state + 1